In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# 建模过程
def modeling(train):
    print("开始建模")
    
    X_train = train.loc[:, train.columns.str.contains('feature')]
    y_train = train.loc[:, 'resp']
    
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=666, test_size=0.2)
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    return model


# 特征工程
def featureEngineer(data):
    data = data[data['weight'] != 0]
    data = data.fillna(-999)
    weight = data['weight'].values
    resp = data['resp'].values
    data['action'] = ((weight * resp) > 0).astype('int')
    return data

    
train = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv", nrows = 10000)
train = featureEngineer(train)
model = modeling(train)
    
    
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set


for (test_df, sample_prediction_df) in iter_test:
    # print(test_df.columns, sample_prediction_df.columns)
    if test_df['weight'].item() > 0:
        X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
        X_test = X_test.fillna(-999)
        # pred_resp = model.predict(X_test)
        # y_preds = (test_df["weight"]*pred_resp > 0).astype('int')
        y_resp = model.predict(X_test)[0]
        y_preds = 0 if y_resp < 0 else 1
    else:
        y_preds = 0
    sample_prediction_df.action = y_preds # THIS LINE
    env.predict(sample_prediction_df)